In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

In [27]:
# cargar la cinética de recuperacion
data_name = 'marvin'
cinetica_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/Daniel_inputs/'
block_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/' + data_name + '.blocks'
prec_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/'+ data_name +'.prec'
cinetica_file = 'REC_MODEL_T240.csv'
cinetica_df = pd.read_csv(cinetica_path + cinetica_file)
blocks_df = pd.read_csv(block_path, header=None)
rec = cinetica_df['rec90'] # panda series para la recuperacion en t=90

In [24]:
# histograma de la recuperacion en el instante 90
hist = cinetica_df.hist(column='rec90', bins=5)
# calcular nuevos valores de la funcion objetivo
# usando la recuperación al dia 90
nblocks = blocks_df.shape[0]
# algunos datos dados
c_conv = 2204.62
rec_AU = 0.88
price_AU = 12
seilling_AU = 0.2
rec_CU = 0.77
price_CU = 20
seilling_CU = 7.2
proc_cost = 4
mine_cost = 0.9 # por tonn
bv_list_cinetic = list()
bv_list_calc = list()
economic_value_list = list()
economic_value_rec_list = list()
with open('marvin_cinetica.upit', 'w') as output, open(block_path, 'r') as bm:
    output.write('NAME: marvin\nTYPE: UPIT\nNBLOCKS: 53271\nOBJECTIVE_FUNCTION: \n')
    for i in range(nblocks):
        #recuperar datos de bm para calcular valor de cada bloque
        linea_bm = bm.readline()
        linea_lista = linea_bm.split()
        tonn = float(linea_lista[4])
        AU = float(linea_lista[5])
        CU = float(linea_lista[6])
        economic_value = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec[i]/100*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_rec = (AU*rec_AU*(price_AU-seilling_AU) + CU*rec_CU*(price_CU-seilling_CU)-(mine_cost+proc_cost))*tonn
        economic_value_list.append(economic_value)
        economic_value_rec_list.append(economic_value_rec)

In [20]:
# optimo con recuperacion del ultimo dia 
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 3052245.7535
Found heuristic solution: objective 2.155104e+07
Found heuristic solution: objective 3.447631e+07
Presolve removed 587607 rows and 46800 columns (presolve time = 5s) ...
Presolve removed 617931 rows and 49095 columns
Presolve time: 6.49s
Presolved: 32700 rows, 4176 columns, 65400 nonzeros
Found heuristic solution: objective 8.477468e+08
Variable types: 0 continuous, 4176 integer (4176 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5500000e+09   0.000000e+00   4.745000e+03      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e+00      7s
    5515    1.2375244e+09   0.000000e+00   0.000000e

In [25]:
# optimo con recuperacion cte
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(quicksum(economic_value_rec_list[i]*x[i] for i in range(nblocks)), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()

Optimize a model with 650631 rows, 53271 columns and 1301262 nonzeros
Variable types: 0 continuous, 53271 integer (53271 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 2796059.4414
Found heuristic solution: objective 2.031152e+07
Found heuristic solution: objective 3.235205e+07
Presolve removed 584413 rows and 46463 columns (presolve time = 5s) ...
Presolve removed 614385 rows and 48755 columns
Presolve time: 6.56s
Presolved: 36246 rows, 4516 columns, 72492 nonzeros
Found heuristic solution: objective 7.999334e+08
Variable types: 0 continuous, 4516 integer (4516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5643467e+09   0.000000e+00   5.157000e+03      7s
    5659    1.2406584e+09   0.000000e+00   0.000000e+00      7s
    5659    1.2406584e+09   0.000000e+00   0.000000e

In [28]:
# obtener funcion objetivo del .pcpsp, para comparar
# el resultado del modelo gurobi con el modelo AMPL
data_name = 'sm2'
pcpsp_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/' + data_name + '.pcpsp'
prec_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/' + data_name + '.prec'
upit_path = 'C:/Users/Equipo/Desktop/Memoria/Codigos/minelib_inputs/'+ data_name + '.upit'
objective_function_pcpsp = {}
resource_constraint_ub_limits = {}
resource_constraint_lb_limits = {}
resource_constraint_coefficients = {}
with open(pcpsp_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        if linea_lista[0] == 'NAME:':
            dato = linea_lista[1].strip('\n')
            name = dato
        elif linea_lista[0] == 'NBLOCKS:':
            dato = linea_lista[1].strip('\n')
            nblocks = int(dato)
        elif linea_lista[0] == 'NPERIODS:':
            dato = linea_lista[1].strip('\n')
            nperiods = int(dato)
        elif linea_lista[0] == 'NDESTINATIONS:':
            dato = linea_lista[1].strip('\n')
            ndestinations = int(dato)
        elif linea_lista[0] == 'NRESOURCE_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            nresource_side_constraints = int(dato)
        elif linea_lista[0] == 'NGENERAL_SIDE_CONSTRAINTS:':
            dato = linea_lista[1].strip('\n')
            ngeneral_side_constraints = int(dato)
        elif linea_lista[0] == 'DISCOUNT_RATE:':
            dato = linea_lista[1].strip('\n')
            discount_rate = float(dato)
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_LIMITS:':
            for r in range(nresource_side_constraints):
                for t in range(nperiods):
                    linea = f.readline()
                    lista = linea.split()
                    resource_constraint_ub_limits[r,t] = int(lista[-1])
                    resource_constraint_lb_limits[r,t] = '-Infinity' # falta hacer para el caso general
    
        elif linea_lista[0] == 'OBJECTIVE_FUNCTION:':
            for b in range(nblocks):
                linea = f.readline()
                lista= linea.split()
                objective_function_pcpsp[b,0] = float(lista[1])
                objective_function_pcpsp[b,1] = float(lista[2])
        elif linea_lista[0] == 'RESOURCE_CONSTRAINT_COEFFICIENTS:':
                for linea in f:
                    if linea == 'EOF\n':
                        break
                    lista = linea.split()
                    b = int(lista[0])
                    d = int(lista[1])
                    r = int(lista[2])
                    resource_constraint_coefficients[b,r,d] = float(lista[3])

# llenar con ceros las entradas de resource_constraint_coefficients
# que no está definidas
for b,r,d in itertools.product(range(nblocks), range(nresource_side_constraints),range(ndestinations)):
    if not (b,r,d) in resource_constraint_coefficients:
        resource_constraint_coefficients[b,r,d] = 0

print('Data base name: %s' % (name))
print('NBLOCKS: %d' % nblocks)
print('NPERIODS: %d' % nperiods)
print('NDESTINATIONS: %d' % ndestinations)
print('NRESOURCE_SIDE_CONSTRAINTS: %d' % nresource_side_constraints)
print('NGENERAL_SIDE_CONSTRAINTS: %d' % ngeneral_side_constraints)
print('DISCOUNT_RATE: %.2f' % discount_rate)

Data base name: SM2
NBLOCKS: 99014
NPERIODS: 30
NDESTINATIONS: 2
NRESOURCE_SIDE_CONSTRAINTS: 2
NGENERAL_SIDE_CONSTRAINTS: 0
DISCOUNT_RATE: 0.10


In [29]:
bv_list = list() # block value list
with open(upit_path, 'r') as f:
    for i in range(4):
        f.readline()
    for line in f:
        if not line == 'EOF\n':
            lista = line.split()
            bv_list.append(float(lista[1]))

In [30]:
# beneficio con valores objetivos dado por .upit
m = Model()
# variable de desicion para el modelo
x = {}
for i in range(nblocks):
    x[i] = m.addVar(vtype=GRB.BINARY, name = "x%d" % i)
m.update()
# definir objetivo
m.setObjective(LinExpr([bv_list[i]for i in range(nblocks)], [x[i] for i in range(nblocks)]), GRB.MAXIMIZE)
# definir restricciones
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        u = int(linea_lista[0])
        for j in range(nvecinos):
            v = int(linea_lista[j+2])
            m.addConstr(x[u] <= x[v])
m.optimize()
# recuperar upit
blocks_id = [i for i in range(nblocks) if x[i].x==1] # recuperar upit
print('UPIT BLOCKS: %d' % len(blocks_id))

Optimize a model with 96642 rows, 99014 columns and 193284 nonzeros
Variable types: 0 continuous, 99014 integer (99014 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 3.921180e+08
Presolve removed 96642 rows and 99014 columns
Presolve time: 0.21s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.24 seconds
Thread count was 1 (of 16 available processors)

Solution count 2: 2.7436e+09 3.92118e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.743603730300e+09, best bound 2.743603730300e+09, gap 0.0000%
UPIT BLOCKS: 18388


In [31]:
# pcpsp model: variables y restricciones
m = Model()
# definir variables para pcpsp
x_pcpsp = {}
y = {}
blocks = blocks_id
for i in blocks:
    for t in range(nperiods):
        x_pcpsp[i,t] = m.addVar(vtype=GRB.BINARY, name = "x_pcpsp%d" % i)
for i in blocks:
    for d in range(ndestinations):
        for t in range(nperiods):
            y[i,d,t] = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS)

indices = list(itertools.product(range(nperiods), range(ndestinations),blocks))
objetivo_expr = LinExpr([(1.0/(1.0 + discount_rate))**t * objective_function_pcpsp[b,d] for t,d,b in indices],
                   [y[b,d,t] for t,d,b in indices])
m.setObjective(objetivo_expr, GRB.MAXIMIZE)
for r in range(nresource_side_constraints):
    for t in range(nperiods):
        blockTimesDest = list(itertools.product(blocks,range(ndestinations)))
        resource = LinExpr([resource_constraint_coefficients[b,r,d] for b,d in blockTimesDest], 
                           [y[b,d,t] for b,d in blockTimesDest])
        m.addConstr(resource_constraint_lb_limits[r,t] <=
                   resource <= resource_constraint_ub_limits[r,t])
# CliqueBlock
for b in blocks:
    m.addConstr(LinExpr([1 for t in range(nperiods)], [x_pcpsp[b,t] for t in range(nperiods)])<=1)
# SumDest
for b in blocks:
    for t in range(nperiods):
        m.addConstr(LinExpr([1 for d in range(ndestinations)], [y[b,d,t] for d in range(ndestinations)])==x_pcpsp[b,t])
# Precedence
with open(prec_path, 'r') as f:
    for linea in f:
        linea_lista = linea.split()
        nvecinos = int(linea_lista[1])
        b = int(linea_lista[0])
        if b in blocks: # solo agregar precedencia para b in UPIT
            for j in range(nvecinos):
                v = int(linea_lista[j+2])
                for t in range(nperiods):
                    m.addConstr(LinExpr([1 for s in range(t+1)], [x_pcpsp[b,s] for s in range(t+1)]) <= LinExpr( [1 for s in range(t+1)], [x_pcpsp[v,s] for s in range(t+1)]))

In [ ]:
m.optimize()

Optimize a model with 1096648 rows, 1654920 columns and 20000730 nonzeros
Variable types: 1103280 continuous, 551640 integer (551640 binary)
Coefficient statistics:
  Matrix range     [8e-01, 3e+04]
  Objective range  [9e-02, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 232890 columns (presolve time = 5s) ...
Presolve removed 232890 rows and 465780 columns (presolve time = 10s) ...
Presolve removed 232890 rows and 465780 columns (presolve time = 15s) ...
Presolve removed 232920 rows and 784530 columns (presolve time = 22s) ...
Presolve removed 551670 rows and 1103280 columns (presolve time = 25s) ...
Presolve removed 551670 rows and 1103280 columns (presolve time = 32s) ...
Presolve removed 551670 rows and 1103280 columns (presolve time = 35s) ...
Presolve removed 568402 rows and 1103280 columns (presolve time = 44s) ...
Presolve removed 568402 rows and 1103280 columns (presolve tim

  188056    2.3263632e+09   3.119000e+08   0.000000e+00    604s
  189266    2.3225456e+09   1.438362e+08   0.000000e+00    609s
  190016    2.3204818e+09   2.630546e+08   0.000000e+00    613s
  191376    2.3162978e+09   4.493952e+08   0.000000e+00    619s
  192216    2.3141027e+09   1.318486e+09   0.000000e+00    626s
  193236    2.3103407e+09   1.217419e+09   0.000000e+00    631s
  193496    2.3102954e+09   1.224746e+09   0.000000e+00    637s
  195206    2.3016583e+09   9.453096e+08   0.000000e+00    643s
  196626    2.2953132e+09   8.563105e+08   0.000000e+00    648s
  198276    2.2849154e+09   1.332182e+09   0.000000e+00    654s
  199986    2.2739167e+09   1.335430e+09   0.000000e+00    659s
  201006    2.2701638e+09   4.384076e+08   0.000000e+00    664s
  202186    2.2663717e+09   3.859854e+08   0.000000e+00    669s
  203576    2.2606905e+09   2.828476e+08   0.000000e+00    676s
  204506    2.2569112e+09   2.695488e+08   0.000000e+00    681s
  205486    2.2537745e+09   2.599040e+08

  287766    2.0803215e+09   9.456192e+07   0.000000e+00   1427s
  288286    2.0799654e+09   4.933263e+07   0.000000e+00   1437s
  288856    2.0794874e+09   5.935554e+07   0.000000e+00   1446s
  289266    2.0792357e+09   1.207523e+08   0.000000e+00   1456s
  289836    2.0785271e+09   9.574404e+07   0.000000e+00   1463s
  290266    2.0781862e+09   5.884941e+08   0.000000e+00   1470s
  290746    2.0777648e+09   1.173493e+08   0.000000e+00   1475s
  291276    2.0770801e+09   1.862489e+08   0.000000e+00   1480s
  292176    2.0739572e+09   1.680505e+08   0.000000e+00   1487s
  293006    2.0725181e+09   5.083558e+07   0.000000e+00   1496s
  293616    2.0719578e+09   1.519111e+08   0.000000e+00   1501s
  294246    2.0711251e+09   5.379417e+07   0.000000e+00   1507s
  294936    2.0702715e+09   1.037801e+08   0.000000e+00   1516s
  295476    2.0696792e+09   5.023594e+07   0.000000e+00   1520s
  296156    2.0688785e+09   3.523539e+08   0.000000e+00   1527s
  296926    2.0677284e+09   8.487289e+07

  390046    1.9325085e+09   1.098330e+08   0.000000e+00   2430s
  390566    1.9323321e+09   1.564238e+08   0.000000e+00   2441s
  391196    1.9317381e+09   8.731242e+07   0.000000e+00   2453s
  391726    1.9311994e+09   1.133338e+08   0.000000e+00   2459s
  392306    1.9307065e+09   9.116088e+07   0.000000e+00   2470s
  393136    1.9296893e+09   6.422918e+07   0.000000e+00   2477s
  393476    1.9296804e+09   4.744604e+07   0.000000e+00   2489s
  394056    1.9292958e+09   5.763065e+07   0.000000e+00   2500s
  394566    1.9288374e+09   2.954216e+08   0.000000e+00   2510s
  394826    1.9288216e+09   1.842502e+08   0.000000e+00   2516s
  395516    1.9282871e+09   1.106108e+08   0.000000e+00   2525s
  396456    1.9276184e+09   2.189739e+08   0.000000e+00   2531s
  397106    1.9272861e+09   5.509984e+08   0.000000e+00   2541s
  397616    1.9271781e+09   2.639262e+08   0.000000e+00   2552s
  398366    1.9266837e+09   6.108799e+08   0.000000e+00   2563s
  399066    1.9264387e+09   7.253711e+08

  512886    1.7363493e+09   1.738500e+09   0.000000e+00   3624s
  513166    1.7361940e+09   1.775025e+09   0.000000e+00   3640s
  513986    1.7356487e+09   1.701156e+09   0.000000e+00   3647s
  514756    1.7350927e+09   1.689228e+09   0.000000e+00   3655s
  515546    1.7344843e+09   1.660838e+09   0.000000e+00   3664s
  516316    1.7338349e+09   8.444421e+08   0.000000e+00   3673s
  517016    1.7333532e+09   1.549470e+09   0.000000e+00   3682s
  517626    1.7328338e+09   1.938586e+09   0.000000e+00   3692s
  518076    1.7326091e+09   1.386862e+09   0.000000e+00   3704s
  518526    1.7323555e+09   1.406280e+09   0.000000e+00   3714s
  519026    1.7320946e+09   1.374548e+09   0.000000e+00   3725s
  519526    1.7318403e+09   1.379746e+09   0.000000e+00   3736s
  519956    1.7315782e+09   1.306705e+09   0.000000e+00   3746s
  520546    1.7308463e+09   1.347327e+09   0.000000e+00   3757s
  521006    1.7306370e+09   1.292011e+09   0.000000e+00   3769s
  521496    1.7303316e+09   1.307867e+09

  585246    1.6780454e+09   5.457462e+07   0.000000e+00   5027s
  586166    1.6776976e+09   1.815059e+08   0.000000e+00   5039s
  586956    1.6773962e+09   4.759998e+07   0.000000e+00   5050s
  587806    1.6771060e+09   4.116830e+07   0.000000e+00   5061s
  588646    1.6768306e+09   8.805054e+07   0.000000e+00   5071s
  589526    1.6765464e+09   8.382841e+07   0.000000e+00   5081s
  590356    1.6762135e+09   6.646163e+07   0.000000e+00   5091s
  591176    1.6758930e+09   7.345490e+07   0.000000e+00   5099s
  592046    1.6755873e+09   8.255919e+07   0.000000e+00   5108s
  592886    1.6753204e+09   4.400413e+07   0.000000e+00   5120s
  593746    1.6750320e+09   1.002039e+08   0.000000e+00   5132s
  594636    1.6747876e+09   1.074088e+08   0.000000e+00   5140s
  595426    1.6745354e+09   1.212223e+08   0.000000e+00   5151s
  596256    1.6742883e+09   9.092283e+07   0.000000e+00   5161s
  597026    1.6740783e+09   3.320631e+07   0.000000e+00   5173s
  597816    1.6738622e+09   8.728641e+07

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'


  682786    1.6289055e+09   3.304805e+07   0.000000e+00   6193s
  683666    1.6286631e+09   3.672569e+07   0.000000e+00   6201s
  684626    1.6283902e+09   7.188106e+07   0.000000e+00   6214s
  685686    1.6278681e+09   4.760008e+08   0.000000e+00   6226s
  687596    1.6261380e+09   1.660884e+08   0.000000e+00   6232s
  689416    1.6256003e+09   1.562768e+08   0.000000e+00   6238s
  691696    1.6253096e+09   2.170160e+08   0.000000e+00   6244s
  692846    1.6251906e+09   2.173568e+08   0.000000e+00   6251s
  693456    1.6250340e+09   6.098433e+07   0.000000e+00   6259s
  694896    1.6246557e+09   2.379095e+08   0.000000e+00   6267s
  695796    1.6246038e+09   1.662845e+07   0.000000e+00   6276s
  696926    1.6245363e+09   7.836128e+07   0.000000e+00   6286s
  698226    1.6238289e+09   1.396563e+07   0.000000e+00   6295s
  699336    1.6235945e+09   1.934496e+08   0.000000e+00   6305s
  700516    1.6233745e+09   8.204752e+07   0.000000e+00   6313s
  701416    1.6233324e+09   2.942275e+07

In [63]:
# imprimir cuantos bloques remueve por periodo
for t in range(nperiods):
    blocks_id_schedule = [i for i in blocks_id if x_pcpsp[i,t].x==1]
    print(len(blocks_id_schedule))

406
366
287
0
0
0
